## Pregunta 3:
### ¿La distribución del nivel de escolaridad en víctimas de violencia intrafamiliar difiere significativamente de la distribución observada en personas que se divorcian?

IMPORTACIÓN DE LIBRERÍAS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import seaborn as sb
import sklearn.cluster as cluster
import sklearn.metrics as metrics
import sklearn.preprocessing
import scipy.cluster.hierarchy as sch
import skfuzzy as fuzz
import pylab
import sklearn.mixture as mixture
import pyclustertend 
import random
from scipy import stats
import seaborn as sns
from scipy.stats import chi2_contingency

CARGA DE ARCHIVOS

In [3]:
df_div = pd.read_csv('../files/divorcios_limpio1.csv', encoding="latin1")
df_viol = pd.read_csv('../files/violencia_intrafamiliar_limpio1.csv', encoding="latin1")

/var/folders/dc/_dt3fhrs48n2b_4mmncch3440000gp/T/ipykernel_30209/3719126337.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_div = pd.read_csv('../files/divorcios_limpio1.csv', encoding="latin1")
/var/folders/dc/_dt3fhrs48n2b_4mmncch3440000gp/T/ipykernel_30209/3719126337.py:2: DtypeWarning: Columns (1,3,4,5,7,8,9,11,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_viol = pd.read_csv('../files/violencia_intrafamiliar_limpio1.csv', encoding="latin1")


ASIGNACIÓN DE CLASES

In [7]:
def clasificar_escolaridad(valor):
    if pd.isna(valor):
        return pd.NA

    if "PRIMARIA" in valor:
        return "BAJA"
    
    if "BSICO" in valor or "BASICO" in valor or "DIVERSIFICADO" in valor:
        return "MEDIA"
    
    if "UNIVERSITARIO" in valor:
        return "ALTA"
    
    if "NINGUNA" in valor:
        return "BAJA"
    
    return pd.NA

In [ ]:
df_div["ESCOLARIDAD_MUJER_NOMBRE"] = df_div["ESCOLARIDAD_MUJER_NOMBRE"].str.upper()

df_div["ESC_GRUPO_MUJ"] = df_div["ESCOLARIDAD_MUJER_NOMBRE"].apply(clasificar_escolaridad)

In [11]:

df_viol["ESC_GRUPO"] = df_viol["VIC_ESCOLARIDAD"].apply(clasificar_escolaridad)

In [12]:
viol = df_viol["ESC_GRUPO"].value_counts()
muj_div = df_div["ESC_GRUPO_MUJ"].value_counts()

tabla_comp_muj = pd.DataFrame({
    "Violencia": viol,
    "Divorcio_Mujeres": muj_div
}).fillna(0)

print(tabla_comp_muj)

       Violencia  Divorcio_Mujeres
ALTA        2605               0.0
BAJA       91910           20216.0
MEDIA      21124           14084.0
